In [1]:
from timeseries_ml_utils.data import DataFetcher, DataGenerator
from scipy.fftpack import dct, idct
from keras.models import Sequential, load_model
from keras.layers import Input, Dense, GRU, Embedding, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.utils import plot_model
from keras.callbacks import TensorBoard
import keras
import matplotlib.pyplot as plt
import re

Using TensorFlow backend.


In [4]:
# fetch data
data = DataFetcher(["GLD.US"], limit=400)
data.fetch_data().tail()


,GLD.US.Open,GLD.US.High,GLD.US.Low,GLD.US.Close,GLD.US.Volume
Date,,,,,
2018-11-12,114.01,114.05,113.58,113.66,7499046
2018-11-13,113.72,113.93,113.42,113.70,9309074
2018-11-14,113.53,115.06,113.46,114.64,13950218
2018-11-15,114.83,114.99,114.56,114.77,8910585
2018-11-16,115.83,115.88,115.37,115.62,8576955


In [5]:
print(len(data.get_dataframe()))
model_data = DataGenerator(data.get_dataframe(), {"^trigonometric": False, "(Open|High|Low|Close)$": True}, {"GLD.US.Close$": True}, aggregation_window_size=16)
print(model_data.batch_feature_shape)
print(model_data.batch_label_shape)
model_data.features, model_data.labels


400
(100, 260, 224)
(100, 16)


([('GLD.US.Open', True),
  ('GLD.US.High', True),
  ('GLD.US.Low', True),
  ('GLD.US.Close', True),
  ('trigonometric_time.cos_dow', False),
  ('trigonometric_time.sin_dow', False),
  ('trigonometric_time.cos_woy', False),
  ('trigonometric_time.sin_woy', False),
  ('trigonometric_time.cos_doy', False),
  ('trigonometric_time.sin_doy', False),
  ('trigonometric_time.sin_yer', False),
  ('trigonometric_time.cos_yer', False),
  ('trigonometric_time.sin_dec', False),
  ('trigonometric_time.cos_dec', False)],
 [('GLD.US.Close', True)])

In [6]:
model = Sequential()
model.add(LSTM(model_data.batch_label_shape[-1],
               name="LSTM-Layer-1",
               batch_input_shape=model_data.batch_feature_shape,
               activation='tanh',
               dropout=0,
               recurrent_dropout=0,
               stateful=True,
               return_sequences=model_data.return_sequences))

model.compile("Adam", loss="mse")
print(model.layers[0].get_weights()[0][0])
model.save("/tmp/keras-foo-1.h5")

[ 0.08206306 -0.0330242   0.04185155 -0.13412641 -0.09821205  0.09963976
  0.07405743  0.09497845  0.12112373  0.13143334  0.06866664 -0.00357862
 -0.08026917 -0.13141754 -0.12587874  0.13704938 -0.0839461   0.09408696
 -0.00530456  0.06393467  0.03855368 -0.09133528  0.07176223 -0.07212893
  0.01656479 -0.08990645 -0.05447047  0.11742646  0.01304977 -0.03246699
  0.08120254  0.09567341 -0.02984876 -0.138614   -0.06265507 -0.11114818
 -0.08198089  0.05587141  0.01400296 -0.10568072  0.07071674  0.08293326
  0.11936402 -0.10799177  0.10157792  0.02209719 -0.01852545 -0.05530729
  0.07708055  0.04223739  0.0560337  -0.02141678  0.07337326  0.11377952
 -0.08622105 -0.10229588  0.06336744  0.0917377   0.07158762 -0.00859262
  0.13244659  0.12060612  0.03626393  0.12358963]


In [ ]:
model = load_model("/tmp/keras-foo-1.h5")
hist = model.fit_generator(generator=model_data,
                           validation_data=model_data.as_test_data_generator(model),
                           epochs=1,
                           use_multiprocessing=True,
                           workers=4,
                           shuffle=False,
                           callbacks=[TensorBoard(log_dir='./logs')])


Epoch 1/1
 9/20 [============>.................] - ETA: 8s - loss: 0.7369

In [ ]:
td = model_data.as_test_data_generator(model)
td.on_epoch_end()
td.accuracy["epoch"].hist(figsize=(10, 7))